# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [1]:
import requests
import pyowm
import json

api_key = "c788b1883c6f01619e31d654ecb4fe9d"

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [2]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 284.47,
  'feels_like': 283.41,
  'temp_min': 283.01,
  'temp_max': 285.92,
  'pressure': 1007,
  'humidity': 67,
  'sea_level': 1007,
  'grnd_level': 1003},
 'visibility': 10000,
 'wind': {'speed': 4.12, 'deg': 70},
 'clouds': {'all': 40},
 'dt': 1745143660,
 'sys': {'type': 2,
  'id': 268730,
  'country': 'GB',
  'sunrise': 1745124828,
  'sunset': 1745175875},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [3]:
#your code here
from pyowm.owm import OWM

owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
print(weather.status)          # short version of status (eg. 'Rain')
weather.detailed_status  # detailed version of status (eg. 'light rain')

Clouds


'scattered clouds'

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [ ]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s") #I think this is wrong and it is in Farhenheits

Temperature: 285.55°C, Humidity: 64%, Wind Speed: 3.13 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [7]:
#your code here
observation = mgr.weather_at_place(city)
weather = observation.weather

temperature = weather.temperature()['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

# Print the results
print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 285.71°C, Humidity: 64%, Wind Speed: 2.24 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [8]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [9]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-04-20 12:00:00, Temp: 290.92°C
Time: 2025-04-20 15:00:00, Temp: 290.51°C
Time: 2025-04-20 18:00:00, Temp: 292.25°C
Time: 2025-04-20 21:00:00, Temp: 292.43°C
Time: 2025-04-21 00:00:00, Temp: 288.43°C
Time: 2025-04-21 03:00:00, Temp: 286.43°C
Time: 2025-04-21 06:00:00, Temp: 284.85°C
Time: 2025-04-21 09:00:00, Temp: 283.67°C
Time: 2025-04-21 12:00:00, Temp: 284.17°C
Time: 2025-04-21 15:00:00, Temp: 284.14°C
Time: 2025-04-21 18:00:00, Temp: 285.84°C
Time: 2025-04-21 21:00:00, Temp: 286.12°C
Time: 2025-04-22 00:00:00, Temp: 285.31°C
Time: 2025-04-22 03:00:00, Temp: 284.96°C
Time: 2025-04-22 06:00:00, Temp: 285.29°C
Time: 2025-04-22 09:00:00, Temp: 285.87°C
Time: 2025-04-22 12:00:00, Temp: 287.55°C
Time: 2025-04-22 15:00:00, Temp: 291.71°C
Time: 2025-04-22 18:00:00, Temp: 293.76°C
Time: 2025-04-22 21:00:00, Temp: 294.91°C
Time: 2025-04-23 00:00:00, Temp: 292.98°C
Time: 2025-04-23 03:00:00, Temp: 287.8°C
Time: 2025-04-23 06:00:00, Temp: 283.98°C
Time: 2025-04-23 09:00:00, Temp: 28

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [ ]:
#your code here
forecast = mgr.forecast_at_place(city, "3h").forecast

print(forecast)

for weather in forecast:
    print(f"Time:{weather.reference_time('iso')}, Temp: {weather.temperature()['temp']}")

<pyowm.weatherapi25.forecast.Forecast - reception_time=2025-04-20 10:46:56+00:00, interval=3h>
Time:2025-04-20 12:00:00+00:00, Temp: 290.92
Time:2025-04-20 15:00:00+00:00, Temp: 290.51
Time:2025-04-20 18:00:00+00:00, Temp: 292.25
Time:2025-04-20 21:00:00+00:00, Temp: 292.43
Time:2025-04-21 00:00:00+00:00, Temp: 288.43
Time:2025-04-21 03:00:00+00:00, Temp: 286.43
Time:2025-04-21 06:00:00+00:00, Temp: 284.85
Time:2025-04-21 09:00:00+00:00, Temp: 283.67
Time:2025-04-21 12:00:00+00:00, Temp: 284.17
Time:2025-04-21 15:00:00+00:00, Temp: 284.14
Time:2025-04-21 18:00:00+00:00, Temp: 285.84
Time:2025-04-21 21:00:00+00:00, Temp: 286.12
Time:2025-04-22 00:00:00+00:00, Temp: 285.31
Time:2025-04-22 03:00:00+00:00, Temp: 284.96
Time:2025-04-22 06:00:00+00:00, Temp: 285.29
Time:2025-04-22 09:00:00+00:00, Temp: 285.87
Time:2025-04-22 12:00:00+00:00, Temp: 287.55
Time:2025-04-22 15:00:00+00:00, Temp: 291.71
Time:2025-04-22 18:00:00+00:00, Temp: 293.76
Time:2025-04-22 21:00:00+00:00, Temp: 294.91
Time:

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [24]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 285.91°C, Fahrenheit: 546.638°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [25]:
#your code here
from pyowm.owm import OWM

owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather

print(f"Temperature in Celsius: {weather.temperature('celsius')['temp']}°C, Fahrenheit: {weather.temperature('fahrenheit')['temp']}°F")

Temperature in Celsius: 12.77°C, Fahrenheit: 54.99°F
